# Playground - data flow graph 

In [91]:
import os
import networkx as nx
import pyvis as pv
import matplotlib.pyplot as plt
from typing import Iterable, Tuple, Set, List, Dict, OrderedDict
import bril_model as bm
from bril_model.form_blocks import print_blocks, form_blocks

DEMO_BRIL_FILE = "../bril/benchmarks/core/is-decreasing.bril"
# DEMO_BRIL_FILE = "../bril/benchmarks/core/hanoi.bril"
# DEMO_BRIL_FILE = "../bril/benchmarks/core/birthday.bril"

bbs = bm.BrilScript(script_name=os.path.basename(DEMO_BRIL_FILE), file_dir=os.path.dirname(DEMO_BRIL_FILE))

In [92]:
ENTRY_POINT_NAME = 'ENTRY\nPOINT'
RETURN_POINT_NAME = 'RETURN\nPOINT'

def iter_func_blocks(bs: bm.BrilScript) -> Iterable[Tuple[bm.BrilFunction, OrderedDict[bm.BrilInstruction_Label, List[bm.BrilInstruction]]]]:
    for each_func in bs.functions:
        block_dict: OrderedDict[bm.BrilInstruction_Label, List[bm.BrilInstruction]] = {}
        anonymous_id = 0
        for each_block in form_blocks(each_func.instrs):
            this_block_label = None
            if isinstance(each_block[0], bm.BrilInstruction_Label):
                this_block_label = each_block[0]
                block_dict[this_block_label] = each_block[1:]
            else:
                this_block_label = bm.BrilInstruction_Label(dict(label='_f{}._anon{}'.format(str(hash(each_func.name)%1000).zfill(3), anonymous_id)))
                block_dict[this_block_label] = each_block[:]
                anonymous_id += 1
            # print(this_block_label)
            # print("\n".join([str(x) for x in block_dict[this_block_label]]))
            # print()
        yield (each_func, block_dict)

app_graph: Dict[bm.BrilFunction, Tuple[nx.DiGraph, Dict[bm.BrilInstruction_Label, List[bm.BrilInstruction]]]] = {}

for each_func, block_dict in iter_func_blocks(bbs):
    print("Function: {}".format(each_func.name))
    for each_label, each_block in block_dict.items():
        print(each_label)
        print("\n".join([str(x) for x in each_block]))
        print()
    fdg = nx.DiGraph()  # function directed graph
    last_label = None
    is_first = True
    for each_label, each_block in block_dict.items():
        fdg.add_node(each_label.label, data=[each_label]+each_block)
        if last_label:
            fdg.add_edge(last_label.label, each_label.label, data='[fallthrough]')
        elif is_first:
            fdg.add_edge(ENTRY_POINT_NAME, each_label.label, data='[enter]')
            is_first = False
        
        final_instr = each_block[-1]
        if final_instr.op in ['jmp', 'br']:
            for redirect_instr_to_dest in final_instr.labels:
                fdg.add_edge(each_label.label, redirect_instr_to_dest, data=final_instr.to_briltxt())
            last_label = None
        elif final_instr.op in ['ret']:
            fdg.add_edge(each_label.label, RETURN_POINT_NAME, data=final_instr.to_briltxt())
            last_label = None
        else:
            last_label = each_label
    app_graph[each_func] = (fdg, block_dict)

Function: main
BrilInstruction_Label ::	<|._f664._anon0|>
BrilInstruction_ValOp ::	tmp0 (bool) <- [call] is_decreasing 
BrilInstruction_ValOp ::	tmp (bool) <- [id] tmp0 
BrilInstruction_EffOp ::	[print] tmp 

Function: is_decreasing
BrilInstruction_Label ::	<|._f037._anon0|>
BrilInstruction_ValOp ::	tmp (int) <- [id] x 
BrilInstruction_Const ::	tmp1 (int) <- [const] 1
BrilInstruction_Const ::	tmp2 (int) <- [const] -1
BrilInstruction_ValOp ::	tmp3 (int) <- [mul] tmp1, tmp2 
BrilInstruction_ValOp ::	prev (int) <- [id] tmp3 

BrilInstruction_Label ::	<|.label4|>
BrilInstruction_Const ::	tmp7 (int) <- [const] 0
BrilInstruction_ValOp ::	tmp8 (bool) <- [gt] tmp, tmp7 
BrilInstruction_EffOp ::	[br] tmp8 {label5, label6} 

BrilInstruction_Label ::	<|.label5|>
BrilInstruction_ValOp ::	tmp9 (int) <- [call] last_digit 
BrilInstruction_ValOp ::	digit (int) <- [id] tmp9 
BrilInstruction_ValOp ::	tmp10 (bool) <- [lt] digit, prev 
BrilInstruction_EffOp ::	[br] tmp10 {label11, label12} 

BrilInstructi

In [93]:
# for each basic block, generate set of variables and expressions that:
# 1. are used before defined
# 2. are modified in the block
# Mark used before defined variables as set(GEN) and modified variables as set(KILL)
# GEN = {v | v is used before defined}
# KILL = {v | v is modified}


def args_use_before_assign_and_assign(instrs: List[bm.BrilInstruction]) -> Tuple[Set[str],Set[str]]:
    used_first: Set[str] = set()
    written: Set[str] = set()
    for instr in instrs:
        used_first.update(set(instr.args if instr.args else []) - written)
        if instr.dest:
            written.add(instr.dest)
    return used_first, written

def gen_kill_sets(block: List[bm.BrilInstruction]) -> Tuple[Set[str],Set[str]]:
    return args_use_before_assign_and_assign(block)

for each_func, (fdg, block_dict) in app_graph.items():
    print("Function: {}".format(each_func.name))
    for each_node, each_node_data in fdg.nodes(data=True):
        print("Node: {}".format(each_node))
        each_block = each_node_data.get('data', None)
        if each_block:
            _gen, _kill = gen_kill_sets(each_block)
            print("GEN: {}, KILL: {}".format(_gen, _kill))
            fdg.nodes[each_node]['GEN'] = _gen
            fdg.nodes[each_node]['KILL'] = _kill
        print()

Function: main
Node: _f664._anon0
GEN: {'x'}, KILL: {'tmp0', 'tmp'}

Node: ENTRY
POINT

Function: is_decreasing
Node: _f037._anon0
GEN: {'x'}, KILL: {'tmp2', 'tmp1', 'prev', 'tmp3', 'tmp'}

Node: ENTRY
POINT

Node: label4
GEN: {'tmp'}, KILL: {'tmp7', 'tmp8'}

Node: label5
GEN: {'prev', 'tmp'}, KILL: {'tmp9', 'tmp10', 'digit'}

Node: label6
GEN: set(), KILL: {'tmp17'}

Node: label11
GEN: set(), KILL: {'tmp14'}

Node: label12
GEN: set(), KILL: set()

Node: RETURN
POINT

Node: _f037._anon1
GEN: set(), KILL: set()

Node: label13
GEN: {'tmp', 'digit'}, KILL: {'prev', 'tmp', 'tmp16', 'tmp15'}

Function: last_digit
Node: _f506._anon0
GEN: {'x'}, KILL: {'tmp21', 'tmp22', 'tmp18', 'tmp19', 'tmp20'}

Node: ENTRY
POINT

Node: RETURN
POINT



In [97]:
fdg = app_graph.get(bbs.functions[2])[0]

# Plot with pyvis
net = pv.network.Network(
    directed=True,
    neighborhood_highlight=True,
    notebook=True,
    cdn_resources="remote", 
    height="100vh",
)
# net.show_buttons(['nodes', 'edges', 'layout', 'interaction', 'manipulation', 'physics', 'selection', 'renderer']) # Show part 3 in the plot (optional)
net.from_nx(fdg) # Create directly from nx graph
# net.show_buttons(['physics',])
net.options.interaction.hover = True
net.options.physics.solver = 'forceAtlas2Based'

# Traverse the net nodes of PyVis and convert the data
for node in net.nodes:
    if 'data' in node:
        # remove 'data' key from node, and set 'title' key to the string representation of the data
        node['title'] = "\n\u3000".join([obj.to_briltxt() if hasattr(obj, 'to_briltxt') else str(obj) for obj in node.pop('data')])
        node['title'] += "\n\n" + f"GEN: {node.pop('GEN', '')}" + "\n" + f"KILL: {node.pop('KILL', '')}"
        node['shape'] = 'box'
    if node['id'] in (ENTRY_POINT_NAME, RETURN_POINT_NAME):
        node['color'] = 'grey'
        node['shape'] = 'circle'

for edge in net.edges:
    if 'data' in edge:
        edge['label'] = edge.pop('data')

net.save_graph('test.html')

In [95]:
# print(bbs,"\n")
print_blocks(bbs)

BrilFunction ::	main ( x<int> ) -> None: <3 instr>
anonymous block:
  BrilInstruction_ValOp ::	tmp0 (bool) <- [call] is_decreasing 
  BrilInstruction_ValOp ::	tmp (bool) <- [id] tmp0 
  BrilInstruction_EffOp ::	[print] tmp 

BrilFunction ::	is_decreasing ( x<int> ) -> bool: <29 instr>
anonymous block:
  BrilInstruction_ValOp ::	tmp (int) <- [id] x 
  BrilInstruction_Const ::	tmp1 (int) <- [const] 1
  BrilInstruction_Const ::	tmp2 (int) <- [const] -1
  BrilInstruction_ValOp ::	tmp3 (int) <- [mul] tmp1, tmp2 
  BrilInstruction_ValOp ::	prev (int) <- [id] tmp3 
block "label4":
  BrilInstruction_Const ::	tmp7 (int) <- [const] 0
  BrilInstruction_ValOp ::	tmp8 (bool) <- [gt] tmp, tmp7 
  BrilInstruction_EffOp ::	[br] tmp8 {label5, label6} 
block "label5":
  BrilInstruction_ValOp ::	tmp9 (int) <- [call] last_digit 
  BrilInstruction_ValOp ::	digit (int) <- [id] tmp9 
  BrilInstruction_ValOp ::	tmp10 (bool) <- [lt] digit, prev 
  BrilInstruction_EffOp ::	[br] tmp10 {label11, label12} 
block "